In [1]:
import glob

readpath = 'E:/IP/dataset 256/HTC-1-M7 500/*.jpg'

labels = 'E:/IP/Labels/labels.csv'
objectClass = '1'

images = glob.glob(readpath)
labelfile = open(labels,'w')

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()



readpath = 'E:/IP/dataset 256/iPhone-4s 500/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '2'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()



readpath = 'E:/IP/dataset 256/iPhone-6 500/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '3'

images = glob.glob(readpath)
labelfile = open(labels,'a')

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()




readpath = 'E:/IP/dataset 256/LG Nexus 5x 500/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '4'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()



readpath = 'E:/IP/dataset 256/Motorola Droid Maxx 500/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '5'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()

readpath = 'E:/IP/dataset 256/Motorola Nexus 6 500/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '6'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()


readpath = 'E:/IP/dataset 256/Motorola X 500/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '7'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()



readpath = 'E:/IP/dataset 256/Samsung Galaxy Note 4 500/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '8'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()


readpath = 'E:/IP/dataset 256/Samsung Galaxy S4 500/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '9'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()


readpath = 'E:/IP/dataset 256/Sony Nex 7 500/*.jpg'
labels = 'E:/IP/Labels/labels.csv'
objectClass = '10'

images = glob.glob(readpath)
labelfile = open(labels,'a')  

for image in images:
    labelfile.write(image+','+objectClass+'\n')

labelfile.close()

In [3]:
import random
# read the csv file 
labels = 'E:/IP/Labels/labels.csv'
#create a new csv file as shuffled_labels.csv where the image address will be shuffled
shuffled_labels = 'E:/IP/Labels/shuffled_labels.csv'

labelfile = open(labels, "r")
lines = labelfile.readlines()
labelfile.close()
random.shuffle(lines)
# create shuffle csv
shufflefile = open(shuffled_labels, "w") 
shufflefile.writelines(lines)
shufflefile.close()

In [4]:
import numpy as np
import cv2
import pandas
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils


labels = 'E:/IP/Labels/shuffled_labels.csv'
npzfile = 'E:/IP/Labels/labels.npz'

df = pandas.read_csv(labels)

rows = df.iterrows()

X_temp = []
Y_temp = []

for row in rows:
    image = row[1][0]
    img = cv2.imread(image)
    (b, g, r)=cv2.split(img)
    img=cv2.merge([r,g,b])
    imageClass = row[1][1]
    X_temp.append(img)
    Y_temp.append(imageClass)


       
# one hot encoding to represent one num as a array like 2= [0 1 0] in a 3 class cnn model     
encoder = LabelEncoder()
encoder.fit(Y_temp)
encoded_Y = encoder.transform(Y_temp)
Y = np_utils.to_categorical(encoded_Y)

np.savez(npzfile, X_train=X_temp,Y_train=Y)



Using TensorFlow backend.


In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras import backend as K
K.set_image_data_format('channels_last')


seed = 7
np.random.seed(seed)
num_classes = 10 


npzfile = 'E:/IP/Labels/labels.npz'

dataset =  np.load(npzfile)
x_train = dataset['X_train']
y_train = dataset['Y_train']



x = x_train/255 
# normalizing 

def CNN_model():
    # create model
    
    model = Sequential()
    model.add(Convolution2D(64,kernel_size=3, strides=2 , padding='valid', input_shape=(256,256,3),activation= 'relu' ))
    model.add(Convolution2D(64,kernel_size=3, strides=2 , padding='valid', activation= 'relu' ))
    model.add(Convolution2D(32,kernel_size=3, strides=1 , padding='valid', activation= 'relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Dropout(0.3))
    model.add(Flatten()) 
    model.add(Dense(256, activation= 'relu' ))
    #model.add(Dense(64, activation= 'relu' ))
    model.add(Dense(128, activation= 'relu' ))  
    model.add(Dense(num_classes, activation= 'softmax' )) 
    model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    return model
  
    
    
    
model = CNN_model()

model.summary()
# check will store the best model weight which will monitor the validation accuracy not train accuracy
check  = ModelCheckpoint('best.hdf5', monitor = 'val_categorical_accuracy' )
checkpoints = [check]

model.fit(x, y_train, validation_split = 0.3,epochs=15, batch_size=32,verbose=2, callbacks = checkpoints)
    
    

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 127, 127, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 61, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               7373056   
__________

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\envs\tensorflow-gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-e57e30ca4084>", line 55, in <module>
    model.fit(x, y_train, validation_split = 0.3,epochs=15, batch_size=32,verbose=2, callbacks = checkpoints)
  File "C:\Users\User\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py", line 893, in fit
    initial_epoch=initial_epoch)
  File "C:\Users\User\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py", line 1555, in fit
    batch_size=batch_size)
  File "C:\Users\User\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py", line 1413, in _standardize_user_data
    exception_prefix='target')
  File "C:\Users\User\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py", line 154, in _standardize_input_data
    str(array.shape))
ValueError

ValueError: Error when checking target: expected dense_3 to have shape (None, 10) but got array with shape (964, 2)